In [ ]:
%gui qt
%matplotlib qt
from xmcd_projection import *
import pickle, trimesh
structure_file = 'SOL6-1-3.stl'
magnetisation_file = 'SOL6-1-3_uniform_mag.p'

struct = trimesh.load(structure_file)
magnetisation = np.zeros(struct.vertices.shape)
magnetisation[:, 1] = 1

vis = Visualizer(struct, magnetisation)
vis.show()
vis.save_render('test1.png')

In [1]:
%gui qt
%matplotlib qt
from xmcd_projection import *
structure_file = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\Simulations\spirals\models\spiral_det_in_50w30t.msh"
magnetisation_file = r"F:\PhD_data\Simulations\spiral_automotion_simulations\data\basic_automotion\spiral_det_in_50w30t_005\data\data.30.csv"

# import the structure and magnetisation data
points, faces, tetra = get_mesh_data_from_file(structure_file)
magnetisation, _ = get_magnumfe_magnetisation(magnetisation_file)
# define the trimesh from structure data
struct = define_trimesh(points, faces)

# show the visualiser (look at xmcd_visualisation file for details)
vis = Visualizer(struct, magnetisation)

# define the viewing angles (here all default)
vis.set_camera(azi=30, dist=1e5, center=[0, -700, 0])
# show the rendering
vis.show()

In [ ]:
struct

In [ ]:
magnetisation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import os
import xml.etree.ElementTree as ET
import seaborn as sns; sns.set()
import meshio
%gui qt
%matplotlib qt
import numpy as np
from scipy.spatial.transform import Rotation as R
import importlib, sys, os
from xmcd_projection import *
import matplotlib.pyplot as plt

In [ ]:
structure_file = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\Simulations\spirals\models\spiral_det_in_50w30t.msh"

mesh = meshio.read(structure_file)
tetra = [cb.data for cb in mesh.cells if cb.type=='tetra'][0]
points = mesh.points
faces = np.array([np.delete(tr, i) for tr in tetra for i in range(4)])

In [ ]:
np.load

In [ ]:
magnetisation_file = r"F:\PhD_data\Simulations\spiral_automotion_simulations\data\basic_automotion\spiral_det_in_50w30t_005\data\data.30.csv"
data = pd.read_csv(magnetisation_file).rename(columns={'m:0':'mx', 'm:1':'my', 'm:2':'mz', 'Points:0':'x', 'Points:1':'y', 'Points:2':'z'})
data

magnetisation = data.loc[:,['mx', 'my', 'mz']].to_numpy()

In [ ]:
magnetisation2, points2 = get_magnumfe_magnetisation(magnetisation_file)

In [ ]:
magnetisation

In [ ]:
magnetisation2

In [ ]:
pd.read_csv(magnetisation_file)

In [ ]:

points = mesh.points.copy()
points[:,1] *=-1
struct_from_mesh = trimesh.Trimesh(vertices = points, faces = faces, process=False)
# show the visualiser (look at xmcd_visualisation file for details)
vis = Visualizer(struct_from_mesh, -magnetisation)

# define the viewing angles (here all default)
vis.set_camera(azi=30, dist=1e5, center=[0, -700, 0])
# show the rendering
vis.show()

In [ ]:
points2 = data.loc[:,['x', 'y', 'z']].to_numpy()

In [ ]:
np.argmax(np.linalg.norm(points - points2))

In [ ]:
points

In [ ]:
points[:, 10865]

In [ ]:
# define the file with structure and magnetisation
structure_file = 'SOL6-1-3.stl'
magnetisation_file = 'SOL6-1-3_uniform_mag.p'

# load the structure
struct = trimesh.load(structure_file)
with open(magnetisation_file, 'rb') as f:
    magnetisation = pickle.load(f)


In [ ]:
from collections import Counter

def get_faces(tetra):
    # get all the faces
    faces = np.array([np.delete(tr, i) for tr in tetra for i in range(4)])
    faces_sets = [frozenset(fc) for fc in faces]
    return faces_sets

def get_edge_faces(tetra):
    # edge faces are faces that appear in only one tetrahedron
    faces_sets = get_faces(tetra)
    edge_faces = []
    for item, count in Counter(faces_sets).most_common():
        if count == 1:
            edge_faces.append(item)
    return edge_faces

def get_edge_points(edge_faces):
    return set.union(*[set(ef) for ef in edge_faces])

edge_faces = get_edge_faces(tetra)
edge_points = get_edge_points(edge_faces)

edge_coords = np.array([points[i, :] for i in edge_points])

In [ ]:
edge_faces = np.array([list(ef) for ef in edge_faces])

In [ ]:
struct_from_mesh = trimesh.Trimesh(vertices = points, faces = faces, process=False)
magnetisation = np.zeros(points.shape)
magnetisation[:,1] = 1
magnetisation[points[:, 2]<550, :]*=-1

In [ ]:
magnetisation

In [ ]:
np.min(points[:,2])

In [ ]:
points[2, :]

In [ ]:
def get_face_magnetisation(faces, point_mag):
    face_mag = np.zeros(faces.shape)
    for i, f in enumerate(faces):
        face_mag[i, :] = np.mean(point_mag[f, :], axis=1)
    return face_mag

face_mag = get_face_magnetisation(faces, magnetisation)

In [ ]:
face_mag

In [ ]:
faces

In [ ]:
np.mean(magnetisation[faces[0,:], :], axis=1)

In [ ]:
magnetisation[3919, :]

In [ ]:

# show the visualiser (look at xmcd_visualisation file for details)
vis = Visualizer(struct, magnetisation)

# define the viewing angles (here all default)
vis.set_camera(azi=30, dist=1e5, center=[0, -700, 0])
# show the rendering
vis.show()

In [ ]:
vis.set_camera(azi=30, dist=1e5, center=[0, -700, 0])
vis.generate_view()
vis.show()

In [ ]:

# magnetisation = np.zeros(struct_from_mesh.vertices.shape)
# magnetisation[:,1] = 1
# magnetisation[struct_from_mesh.vertices[:, 2]<550, :]*=-1
show_magnetisation(struct_from_mesh, magnetisation, step=10)

In [ ]:
struct_from_mesh.vertices

In [ ]:
struct_from_mesh = trimesh.Trimesh(vertices = points, faces = faces, process=False)
struct_from_mesh.faces

In [ ]:
faces